Item 43 Inherit from collections.abc for Custom Container Types 

Things to Remember
- Inherit directly from Python's container types (like list or dict) for simple use cases.
- Beware of the large number of methods required to implement custom container types correctly.
- Have your custom container types inherit from the interfaces defined in collections.abc to ensure that your classes match required interfaces and behaviors.    

In [ ]:
# - you want to create your own custom list type
#   that has additional methods for counting the
#   frequency of its members 

In [ ]:
# - You get all of list's standard functionality
#   and preserve the semantics familiar to all
#   Python programmers by subclassing list.
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)
    def frequency(self):
        counts = {}
        for item in self:
            counts[item] = counts.get(item, 0) + 1
        return counts

In [ ]:
foo = FrequencyList(['a', 'b', 'a', 'c', 'b', 'a', 'd'])
print('Length is', len(foo))
foo.pop() # - removes and returns 
          #   last object or obj 
          #    from list
print('After pop:', repr(foo))
print('Frequency:', foo.frequency())

In [ ]:
# - you want to provide an object that
#   feels like a list and allows indexing
#   but isn't a list subclass

# base class
class BinaryNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

# now make it act like a sequence
class IndexableNode(BinaryNode):
    def _traverse(self):
        if self.left is not None:
            yield from self.left._traverse() # depth first
        yield self
        if self.right is not None:
            yield from self.right._traverse()
    
     # - special method to implement
     #   to support indexing 
    def __getitem__(self, index):
        for i, item in enumerate(self._traverse()):
            if i == index:
                 return item.value
        raise IndexError(f'Index {index} is out of range')

In [ ]:
tree = IndexableNode(
    10,
    left=IndexableNode(
        5,
        left=IndexableNode(2),
        right=IndexableNode(
            6,
            right=IndexableNode(7))),
    right=IndexableNode(
        15,
        left=IndexableNode(11)))

In [ ]:
print('LRR is', tree.left.right.right.value)
print('Index 0 is', tree[0])
print('Index 1 is', tree[1])
print('11 in the tree', 11 in tree)
print('17 in the tree', 17 in tree)
print('Tree is', list(tree))

looks fine so what's wrong?


In [ ]:
# - implementing __getitem__ alone 
#   isn't enough to provide all of
#   the sequence semantics you'd 
#   expect from a list instance
len(tree) # error 

In [ ]:
class SequenceNode(IndexableNode):
    def __len__(self):
        # - the index starts with 1, and once you are done 
        #   visiting the tree 'count' will have the value of 
        #   the index of the last node.
        for count, _ in enumerate(self._traverse(), 1):
            pass
        return count

In [ ]:
tree = SequenceNode(
    10,
    left=SequenceNode(
        5,
        left=SequenceNode(2),
        right=SequenceNode(
            6,
            right=SequenceNode(7))),
    right=SequenceNode(
        15,
        left=SequenceNode(11)))

print('Tree length is', len(tree))

Unfortunately this is not the end of story
- you are still missing other methods developers would expect to see on a sequence like list or tuple
- count and index, for example, are such methods  

Solution?
- Subclass from Sequence in collections.abc.  
- The module tell you what is wrong in case you forget to implement required methods.   

In [ ]:
from collections.abc import Sequence

In [ ]:
class BadType(Sequence):
    pass

foo = BadType() # error message will tell what is missing

In [ ]:
# - inherit from SequenceNode so you have
#   the impplementaion for __getitem__ and
#   __len__
# - inherit from Seqeunce so you don't need
#   to worry about other required methods
#   such as index and count 
class BetterNode(SequenceNode, Sequence):
    pass


In [ ]:
tree = BetterNode(
    10,
    left=BetterNode(
        5,
        left=BetterNode(2),
        right=BetterNode(
            6,
            right=BetterNode(7))),
    right=BetterNode(
        15,
        left=BetterNode(11)))

In [ ]:
print('Index of 7 is', tree.index(7))
print('Count of 10 is', tree.count(10))

The benefit of using these abstract calsses is even greater for more complex container types such as Set and MutableMapping, which have a large number of special methods that need to be implemented to match Python conventions.